In [1]:
%%capture
%pip install nltk
%pip install boto3

In [16]:
from modeling.modeling_charbert import CharBertModel
from modeling.modeling_roberta import RobertaModel
from run_lm_finetuning import load_and_cache_examples, TextDataset
from modeling.configuration_bert import BertConfig
from modeling.configuration_roberta import RobertaConfig
from transformers import BertTokenizer, RobertaTokenizer
import argparse
import os
import torch
import logging

In [3]:
MODEL_CLASSES = {
    'bert': (BertConfig, CharBertModel, BertTokenizer),
    'roberta': (RobertaConfig, RobertaModel, RobertaTokenizer),
}

In [5]:
# keys = {
#     "output_debug": False,
#     "adv_probability": 0.1,
#     "model_type": model_type,
#     "mlm_probability": 0.1,
#     "block_size": -1,
#     "char_maxlen_for_word": 6,
#     "file_path": "./train_data.txt",
#     "char_vocab": "./data/dict/roberta_char_vocab",
#     "term_vocab": "./data/dict/term_vocab",
#     "data_version": "1",
#     "seed": 42,
#     "input_nraws": 10000
# }

In [6]:
class Args:
    output_debug = False
    adv_probability = 0.1
    model_type = "roberta"
    mlm_probability = 0.1
    block_size = -1
    char_maxlen_for_word = 6
    char_vocab = "./data/dict/roberta_char_vocab"
    term_vocab = "./data/dict/term_vocab"
    data_version = "1"
    seed = 42
    input_nraws = 10000
    config_name = "roberta-large"
    model_name_or_path = "roberta-large"
    tokenizer_name = "roberta-large"

my_args = Args()

In [14]:
def main():
     parser = argparse.ArgumentParser()

     ## Required parameters
     parser.add_argument("--train_data_file", default="./train_data.txt", type=str,
                         help="The input training data file (a text file).")
     parser.add_argument("--output_dir", default="./results/", type=str,
                         help="The output directory where the model predictions and checkpoints will be written.")

     ## Other parameters
     parser.add_argument("--eval_data_file", default=None, type=str,
                         help="An optional input evaluation data file to evaluate the perplexity on (a text file).")

     parser.add_argument("--char_vocab", default="./data/dict/bert_char_vocab", type=str,
                         help="char vocab for charBert")
     parser.add_argument("--term_vocab", default="./data/dict/term_vocab", type=str,
                         help="term vocab for charBert")

     parser.add_argument("--model_type", default="roberta", type=str,
                         help="The model architecture to be fine-tuned.")
     parser.add_argument("--model_name_or_path", default="roberta-large", type=str,
                         help="The model checkpoint for weights initialization.")

     parser.add_argument("--mlm", action='store_true',
                         help="Train with masked-language modeling loss instead of language modeling.")
     parser.add_argument("--mlm_probability", type=float, default=0.10,
                         help="Ratio of tokens to mask for masked language modeling loss")
     parser.add_argument("--adv_probability", type=float, default=0.10,
                         help="Ratio of tokens to mask for masked language modeling loss")

     parser.add_argument("--config_name", default="", type=str,
                         help="Optional pretrained config name or path if not the same as model_name_or_path")
     parser.add_argument("--data_version", default="", type=str,
                         help="training data version for cached file")
     parser.add_argument("--tokenizer_name", default="", type=str,
                         help="Optional pretrained tokenizer name or path if not the same as model_name_or_path")
     parser.add_argument("--cache_dir", default="", type=str,
                         help="Optional directory to store the pre-trained models downloaded from s3 (instread of the default one)")
     parser.add_argument("--block_size", default=-1, type=int,
                         help="Optional input sequence length after tokenization."
                              "The training dataset will be truncated in block of this size for training."
                              "Default to the model max input length for single sentence inputs (take into account special tokens).")
     parser.add_argument("--do_train", action='store_true',
                         help="Whether to run training.")
     parser.add_argument("--do_eval", action='store_true',
                         help="Whether to run eval on the dev set.")
     parser.add_argument("--output_debug", action='store_true',
                         help="Whether to output the debug information.")
     parser.add_argument("--evaluate_during_training", action='store_true',
                         help="Run evaluation during training at each logging step.")
     parser.add_argument("--do_lower_case", action='store_true',
                         help="Set this flag if you are using an uncased model.")
     parser.add_argument("--char_maxlen_for_word", default=6, type=int,
                         help="Max number of char for each word.")

     parser.add_argument("--per_gpu_train_batch_size", default=2, type=int,
                         help="Batch size per GPU/CPU for training.")
     parser.add_argument("--per_gpu_eval_batch_size", default=2, type=int,
                         help="Batch size per GPU/CPU for evaluation.")
     parser.add_argument('--gradient_accumulation_steps', type=int, default=1,
                         help="Number of updates steps to accumulate before performing a backward/update pass.")
     parser.add_argument("--learning_rate", default=5e-5, type=float,
                         help="The initial learning rate for Adam.")
     parser.add_argument("--weight_decay", default=0.0, type=float,
                         help="Weight decay if we apply some.")
     parser.add_argument("--adam_epsilon", default=1e-8, type=float,
                         help="Epsilon for Adam optimizer.")
     parser.add_argument("--max_grad_norm", default=1.0, type=float,
                         help="Max gradient norm.")
     parser.add_argument("--num_train_epochs", default=1.0, type=float,
                         help="Total number of training epochs to perform.")
     parser.add_argument("--max_steps", default=-1, type=int,
                         help="If > 0: set total number of training steps to perform. Override num_train_epochs.")
     parser.add_argument("--warmup_steps", default=0, type=int,
                         help="Linear warmup over warmup_steps.")

     parser.add_argument('--logging_steps', type=int, default=50,
                         help="Log every X updates steps.")
     parser.add_argument('--save_steps', type=int, default=50,
                         help="Save checkpoint every X updates steps.")
     parser.add_argument("--input_nraws", default=10000, type=int,
                         help="number of lines when read the input data each time.")
     parser.add_argument('--save_total_limit', type=int, default=None,
                         help='Limit the total amount of checkpoints, delete the older checkpoints in the output_dir, does not delete by default')
     parser.add_argument("--eval_all_checkpoints", action='store_true',
                         help="Evaluate all checkpoints starting with the same prefix as model_name_or_path ending and ending with step number")
     parser.add_argument("--no_cuda", action='store_true',
                         help="Avoid using CUDA when available")
     parser.add_argument('--overwrite_output_dir', action='store_true',
                         help="Overwrite the content of the output directory")
     parser.add_argument('--overwrite_cache', action='store_true',
                         help="Overwrite the cached training and evaluation sets")
     parser.add_argument('--seed', type=int, default=42,
                         help="random seed for initialization")

     # parser.add_argument('--fp16', action='store_true',
     #                     help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit")
     # parser.add_argument('--fp16_opt_level', type=str, default='O1',
     #                     help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
     #                          "See details at https://nvidia.github.io/apex/amp.html")
     parser.add_argument("--local_rank", type=int, default=-1,
                         help="For distributed training: local_rank")
     parser.add_argument('--server_ip', type=str, default='', help="For distant debugging.")
     parser.add_argument('--server_port', type=str, default='', help="For distant debugging.")
     args, unknown = parser.parse_known_args()

     if args.model_type in ["bert", "roberta", "distilbert", "camembert"] and not args.mlm:
          raise ValueError("BERT and RoBERTa do not have LM heads but masked LM heads. They must be run using the --mlm "
                              "flag (masked language modeling).")
     if args.eval_data_file is None and args.do_eval:
          raise ValueError("Cannot do evaluation without an evaluation data file. Either supply a file to --eval_data_file "
                              "or remove the --do_eval argument.")

     if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train and not args.overwrite_output_dir:
          raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args.output_dir))

     # Setup distant debugging if needed
     if args.server_ip and args.server_port:
          # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
          import ptvsd
          print("Waiting for debugger attach")
          ptvsd.enable_attach(address=(args.server_ip, args.server_port), redirect_output=True)
          ptvsd.wait_for_attach()

     # Setup CUDA, GPU & distributed training
     """
     if args.local_rank == -1 or args.no_cuda:
          device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
          args.n_gpu = torch.cuda.device_count()
     else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
          torch.cuda.set_device(args.local_rank)
          device = torch.device("cuda", args.local_rank)
          torch.distributed.init_process_group(backend='nccl')
          args.n_gpu = 1
     args.device = device
     """
     device = torch.device("cpu")
     args.n_gpu = 0  # Since we're not using GPU
     args.device = device

     # Setup logging
     # logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
     #                     datefmt = '%m/%d/%Y %H:%M:%S',
     #                     level = logging.INFO if args.local_rank in [-1, 0] else logging.WARN)
     # logger.warning("Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
     #                     args.local_rank, device, args.n_gpu, bool(args.local_rank != -1), args.fp16)

     # Set seed
     set_seed(args)

     # Load pretrained model and tokenizer
     if args.local_rank not in [-1, 0]:
          torch.distributed.barrier()  # Barrier to make sure only the first process in distributed training download model & vocab

     config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
     config = config_class.from_pretrained(args.config_name if args.config_name else args.model_name_or_path,
                                             cache_dir=args.cache_dir if args.cache_dir else None)
     tokenizer = tokenizer_class.from_pretrained(args.tokenizer_name if args.tokenizer_name else args.model_name_or_path,
                                                  do_lower_case=args.do_lower_case,
                                                  cache_dir=args.cache_dir if args.cache_dir else None)
     if args.block_size <= 0:
          args.block_size = tokenizer.max_len_single_sentence  # Our input block size will be the max possible for the model
     args.block_size = min(args.block_size, tokenizer.max_len_single_sentence)
     model = model_class.from_pretrained(args.model_name_or_path,
                                             from_tf=bool('.ckpt' in args.model_name_or_path),
                                             config=config,
                                             cache_dir=args.cache_dir if args.cache_dir else None)
     model.to(args.device)

     if args.local_rank == 0:
          torch.distributed.barrier()  # End of barrier to make sure only the first process in distributed training download model & vocab

     # logger.info("Training/evaluation parameters %s", args)

     # Training
     if args.do_train:
          if args.local_rank not in [-1, 0]:
               torch.distributed.barrier()  # Barrier to make sure only the first process in distributed training process the dataset, and the others will use the cache

          train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False)

          if args.local_rank == 0:
               torch.distributed.barrier()

          global_step, tr_loss = train(args, train_dataset, model, tokenizer)
          logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)


     # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
     if args.do_train and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
          # Create output directory if needed
          if not os.path.exists(args.output_dir) and args.local_rank in [-1, 0]:
               os.makedirs(args.output_dir)

          logger.info("Saving model checkpoint to %s", args.output_dir)
          # Save a trained model, configuration and tokenizer using `save_pretrained()`.
          # They can then be reloaded using `from_pretrained()`
          model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
          model_to_save.save_pretrained(args.output_dir)
          tokenizer.save_pretrained(args.output_dir)

          # Good practice: save your training arguments together with the trained model
          torch.save(args, os.path.join(args.output_dir, 'training_args.bin'))

          # Load a trained model and vocabulary that you have fine-tuned
          model = model_class.from_pretrained(args.output_dir)
          tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
          model.to(args.device)


     # Evaluation
     results = {}
     if args.do_eval and args.local_rank in [-1, 0]:
          checkpoints = [args.output_dir]
          if args.eval_all_checkpoints:
               checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + '/**/' + WEIGHTS_NAME, recursive=True)))
               logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
          logger.info("Evaluate the following checkpoints: %s", checkpoints)
          for checkpoint in checkpoints:
               global_step = checkpoint.split('-')[-1] if len(checkpoints) > 1 else ""
               prefix = checkpoint.split('/')[-1] if checkpoint.find('checkpoint') != -1 else ""
               
               model = model_class.from_pretrained(checkpoint)
               model.to(args.device)
               result = evaluate(args, model, tokenizer, prefix=prefix)
               result = dict((k + '_{}'.format(global_step), v) for k, v in result.items())
               results.update(result)

     return results

In [15]:
main()

ValueError: BERT and RoBERTa do not have LM heads but masked LM heads. They must be run using the --mlm flag (masked language modeling).

In [30]:
# def main():
#      parser = argparse.ArgumentParser()
#      parser.add_argument("--model_type", default="roberta", type=str,
#                          help="The model architecture to be fine-tuned.")
#      parser.add_argument("--model_name_or_path", default="roberta-large", type=str,
#                          help="The model checkpoint for weights initialization.")
#      parser.add_argument("--config_name", default="roberta-large", type=str,
#                          help="Optional pretrained config name or path if not the same as model_name_or_path")
#      parser.add_argument("--tokenizer_name", default="roberta-large", type=str,
#                          help="Optional pretrained tokenizer name or path if not the same as model_name_or_path")
#      parser.add_argument("--output_debug", default=False, action='store_true',
#                          help="Whether to output the debug information.")
#      parser.add_argument("--adv_probability", type=float, default=0.10,
#                          help="Ratio of tokens to mask for masked language modeling loss")
#      parser.add_argument("--mlm_probability", type=float, default=0.10,
#                          help="Ratio of tokens to mask for masked language modeling loss")
#      parser.add_argument("--block_size", default=-1, type=int,
#                          help="Optional input sequence length after tokenization."
#                               "The training dataset will be truncated in block of this size for training."
#                               "Default to the model max input length for single sentence inputs (take into account special tokens).")
#      parser.add_argument("--char_maxlen_for_word", default=6, type=int,
#                          help="Max number of char for each word.")
#      parser.add_argument("--char_vocab", default="./data/dict/bert_char_vocab", type=str,
#                          help="char vocab for charBert")
#      parser.add_argument("--term_vocab", default="./data/dict/term_vocab", type=str,
#                          help="term vocab for charBert")
#      parser.add_argument("--data_version", default="", type=str,
#                          help="training data version for cached file")
#      parser.add_argument('--seed', type=int, default=42,
#                          help="random seed for initialization")
#      parser.add_argument("--input_nraws", default=10000, type=int,
#                          help="number of lines when read the input data each time.")
#      parser.add_argument("--train_data_file", default="./train_data.txt", type=str,
#                          help="The input training data file (a text file).")
#      parser.add_argument("--eval_data_file", default="./train_data.txt", type=str,
#                          help="An optional input evaluation data file to evaluate the perplexity on (a text file).")
     
#      args, unknown = parser.parse_known_args()

#      config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
#      config = config_class.from_pretrained(args.config_name, cache_dir=None)
#      model = model_class.from_pretrained(args.model_name_or_path,
#                                         from_tf=False,
#                                         config=config,
#                                         cache_dir=None)

#      tokenizer = tokenizer_class.from_pretrained(args.tokenizer_name)
#      train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False)
     
#      return

In [29]:
main()

cls: <class 'modeling.configuration_roberta.RobertaConfig'>
pretrained_model_name_or_path: roberta-large
pretrained_model_name_or_path: roberta-large
cls.pretrained_config_archive_map: {'roberta-base': 'https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json', 'roberta-large': 'https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-config.json', 'roberta-large-mnli': 'https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-mnli-config.json', 'distilroberta-base': 'https://s3.amazonaws.com/models.huggingface.co/bert/distilroberta-base-config.json', 'roberta-base-openai-detector': 'https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-openai-detector-config.json', 'roberta-large-openai-detector': 'https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-openai-detector-config.json'}


In [15]:
def get_dataset(tokenizer, keys, evaluate=False):
    dataset = TextDataset(tokenizer, **keys)
    return dataset

In [16]:
train_dataset = get_dataset(tokenizer, keys, evaluate=False)

TypeError: __init__() got an unexpected keyword argument 'output_debug'

In [ ]:
outputs = self.bert(char_input_ids,
                            start_ids,
                            end_ids,
                            input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask,
                            inputs_embeds=inputs_embeds,
                            encoder_hidden_states=encoder_hidden_states,
                            encoder_attention_mask=encoder_attention_mask)